In [1]:
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import edhec_risk_kit as erk
import pandas as pd
import numpy as np
import config
%load_ext autoreload
%autoreload 2

In [2]:
all_data = erk.get_df(filename=config.fileName,format=config.dateFormat, mode=None, to_per=True)
all_data.columns = all_data.columns.str.replace(' ', '_')

In [20]:
plot_data = {}
plot_data.update(Real_Estate=all_data['Real_Estate'])

In [21]:
wealth_data_plots, risk_info = erk.get_wealth_index_risk(plot_data)

In [22]:
pyo.iplot(wealth_data_plots)
risk_info

,Real_Estate
ann_ret,0.001106
ann_vol,0.002806
sharpe_ratio,-10.297409
semi_dev,0.000718
Kurtosis,9.107918
Skew,-0.201885
VaR,-0.001181
CVaR,-0.002395
Drawdown,-0.007561


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     59.99
Date:                Thu, 30 Jul 2020   Prob (F-statistic):           1.63e-46
Time:                        14:56:26   Log-Likelihood:                 2580.8
No. Observations:                 405   AIC:                            -5150.
Df Residuals:                     399   BIC:                            -5126.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3103      0.051      6.036      0.000       0.209       0.411
US_Treasuries            1.2125      0.121     10.027      0.000       0.975       1.450
Bond_Risk_Premium        1.0542      0.103     10.208      0.000       0.851       1.257
Inflation_Protection     0.2879      0.129      2.229      0.026       0.034       0.542
Currency_Protection      0.3618      0.173      2.090      0.037       0.021       0.702
Alpha                -3.587e-05   2.28e-05     -1.575      0.116   -8.06e-05     8.9e-06
==============================================================================
Omnibus:                       81.133   Durbin-Watson:                   2.289
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1295.204
Skew:                           0.254   Prob(JB):                    5.63e-282
Kurtosis:                      11.746   Cond. No.                     8.43e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [3]:
# adding noise

np.random.seed(1245)
n_periods = all_data.shape[0]
base = np.reshape(np.array(all_data['World_Equities']), (n_periods, 1))
noise = np.random.normal(loc=base, scale=base.std()*2, size=(n_periods,1))
all_data['Noise'] = noise

In [6]:
explanatory_df_noise = all_data[config.factorName_Noise]
dependent_df_noise = all_data[config.asset_class]
regression_result_noise = erk.regress(dependent_df_noise, explanatory_df_noise, excess_mkt=False, rfcol=config.rfcol)

In [7]:
regression_result_noise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     49.94
Date:                Fri, 31 Jul 2020   Prob (F-statistic):           1.17e-45
Time:                        12:37:47   Log-Likelihood:                 2580.9
No. Observations:                 405   AIC:                            -5148.
Df Residuals:                     398   BIC:                            -5120.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3231      0.057      5.646      0.000       0.211       0.436
US_Treasuries            1.2140      0.121     10.027      0.000       0.976       1.452
Bond_Risk_Premium        1.0542      0.103     10.198      0.000       0.851       1.257
Inflation_Protection     0.2835      0.130      2.187      0.029       0.029       0.538
Currency_Protection      0.3620      0.173      2.088      0.037       0.021       0.703
Noise                   -0.0106      0.021     -0.512      0.609      -0.051       0.030
Alpha                -3.654e-05   2.28e-05     -1.600      0.110   -8.14e-05    8.35e-06
==============================================================================
Omnibus:                       81.023   Durbin-Watson:                   2.286
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1280.814
Skew:                           0.258   Prob(JB):                    7.49e-279
Kurtosis:                      11.697   Cond. No.                     8.44e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# regime changes
normal_mask = all_data['US_Equities'] > 0
crash_mask = all_data['US_Equities'] < 0

explanatory_df_all = all_data[config.factorName]
dependent_df_all = all_data[config.asset_class]

explanatory_df_normal = explanatory_df_all[normal_mask]
dependent_df_normal = dependent_df_all[normal_mask]

explanatory_df_crash = explanatory_df_all[crash_mask]
dependent_df_crash = dependent_df_all[crash_mask]

regression_result_all = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol)
regression_result_normal = erk.regress(dependent_df_normal, explanatory_df_normal, excess_mkt=False, rfcol=config.rfcol)
regression_result_crash = erk.regress(dependent_df_crash, explanatory_df_crash, excess_mkt=False, rfcol=config.rfcol)

In [17]:
regression_result_all.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     59.99
Date:                Fri, 31 Jul 2020   Prob (F-statistic):           1.63e-46
Time:                        14:48:21   Log-Likelihood:                 2580.8
No. Observations:                 405   AIC:                            -5150.
Df Residuals:                     399   BIC:                            -5126.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3103      0.051      6.036      0.000       0.209       0.411
US_Treasuries            1.2125      0.121     10.027      0.000       0.975       1.450
Bond_Risk_Premium        1.0542      0.103     10.208      0.000       0.851       1.257
Inflation_Protection     0.2879      0.129      2.229      0.026       0.034       0.542
Currency_Protection      0.3618      0.173      2.090      0.037       0.021       0.702
Alpha                -3.587e-05   2.28e-05     -1.575      0.116   -8.06e-05     8.9e-06
==============================================================================
Omnibus:                       81.133   Durbin-Watson:                   2.289
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1295.204
Skew:                           0.254   Prob(JB):                    5.63e-282
Kurtosis:                      11.746   Cond. No.                     8.43e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
regression_result_normal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     23.54
Date:                Fri, 31 Jul 2020   Prob (F-statistic):           1.53e-19
Time:                        12:41:02   Log-Likelihood:                 1751.9
No. Observations:                 269   AIC:                            -3492.
Df Residuals:                     263   BIC:                            -3470.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.1502      0.067      2.251      0.025       0.019       0.282
US_Treasuries            1.3029      0.144      9.042      0.000       1.019       1.587
Bond_Risk_Premium        1.1006      0.131      8.387      0.000       0.842       1.359
Inflation_Protection     0.2102      0.142      1.477      0.141      -0.070       0.490
Currency_Protection      0.0518      0.196      0.265      0.791      -0.333       0.437
Alpha                  3.48e-05   3.15e-05      1.104      0.271   -2.73e-05    9.69e-05
==============================================================================
Omnibus:                      176.288   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3544.164
Skew:                           2.237   Prob(JB):                         0.00
Kurtosis:                      20.210   Cond. No.                     8.92e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.92e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
regression_result_crash.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.334
Method:                 Least Squares   F-statistic:                     14.46
Date:                Fri, 31 Jul 2020   Prob (F-statistic):           3.10e-11
Time:                        12:41:03   Log-Likelihood:                 838.40
No. Observations:                 135   AIC:                            -1665.
Df Residuals:                     129   BIC:                            -1647.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3678      0.096      3.816      0.000       0.177       0.559
US_Treasuries            0.9523      0.229      4.154      0.000       0.499       1.406
Bond_Risk_Premium        0.7805      0.189      4.129      0.000       0.407       1.154
Inflation_Protection     0.5368      0.263      2.042      0.043       0.017       1.057
Currency_Protection      0.5744      0.349      1.645      0.102      -0.116       1.265
Alpha                   -0.0001   5.03e-05     -2.286      0.024      -0.000   -1.55e-05
==============================================================================
Omnibus:                       33.834   Durbin-Watson:                   1.324
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               84.886
Skew:                          -0.978   Prob(JB):                     3.69e-19
Kurtosis:                       6.356   Cond. No.                     8.32e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
# Lasso
explanatory_df_all = all_data[config.factorName]
dependent_df_all = all_data[config.asset_class]
sk_lasso = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='lasso', lamda=0.1)


        Regression method is lasso
        Time period is 1985-01 - 2018-09
    
       intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
lasso   0.001834        0.349738         0.5422           0.574002   

       Inflation_Protection  Currency_Protection  
lasso                   0.0                  0.0  


In [4]:
#Lasso with noise
explanatory_df_all = all_data[config.factorName_Noise]
dependent_df_all = all_data[config.asset_class]
sk_ridge = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='lasso', lamda=0.1)



        Regression method is lasso
        Time period is 1985-01 - 2018-09
        Alpha is 0.0001234567901234568
        Lambda is 0.1
    
       intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
lasso   0.001834        0.349738         0.5422           0.574002   

       Inflation_Protection  Currency_Protection  Noise  
lasso                   0.0                  0.0    0.0  


In [14]:
# CV_Lasso
explanatory_df_all = all_data[config.factorName]
dependent_df_all = all_data[config.asset_class]
cv_lasso = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='cv_lasso')

Max_alpha is : 0.00030864197530864197

        Regression method is cv_lasso
        Time period is 1985-01 - 2018-09
        Alpha is 6.004239930165855e-05
        Lambda is 0.048634343434343426
    
          intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
cv_lasso  -0.000342        0.311805       0.875641           0.835079   

          Inflation_Protection  Currency_Protection  
cv_lasso              0.044825                  0.0  


In [5]:
# CV_Lasso with noise
explanatory_df_all = all_data[config.factorName_Noise]
dependent_df_all = all_data[config.asset_class]
cv_lasso = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='cv_lasso')



Max_alpha is : 0.00030864197530864197

        Regression method is cv_lasso
        Time period is 1985-01 - 2018-09
        Alpha is 8.17948622022696e-05
        Lambda is 0.06625383838383839
    
          intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
cv_lasso   0.000441        0.324436       0.759458           0.748534   

          Inflation_Protection  Currency_Protection  Noise  
cv_lasso                   0.0                  0.0   -0.0  


In [5]:
#CV_ElasticNet
explanatory_df_all = all_data[config.factorName]
dependent_df_all = all_data[config.asset_class]
cv_lasso = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='cv_elasticnet')


            Best L1 ratio is : 0.9495918775510204
            Best Lasso_Lambda is : 0.050585758887540265            
            Best Ridge_Lambda is : 0.0013426468720193349
        

        Regression method is cv_elasticnet
        Time period is 1985-01 - 2018-09
        Alpha is 6.576673164392461e-05
        Lambda is 0.050585758887540265
    
               intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
cv_elasticnet  -0.000159        0.315769       0.847193           0.813355   

               Inflation_Protection  Currency_Protection  
cv_elasticnet              0.035633                  0.0  


In [6]:
#CV_ElasticNet noise
explanatory_df_all = all_data[config.factorName_Noise]
dependent_df_all = all_data[config.asset_class]
cv_lasso = erk.regress(dependent_df_all, explanatory_df_all, excess_mkt=False, rfcol=config.rfcol, method='cv_elasticnet')


            Best L1 ratio is : 0.99
            Best Lasso_Lambda is : 0.06572245263157893            
            Best Ridge_Lambda is : 0.00033193157894736866
        

        Regression method is cv_elasticnet
        Time period is 1985-01 - 2018-09
        Alpha is 8.195841455490576e-05
        Lambda is 0.06572245263157893
    
               intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
cv_elasticnet    0.00044        0.324609       0.759465           0.748479   

               Inflation_Protection  Currency_Protection  Noise  
cv_elasticnet                   0.0                  0.0   -0.0  


In [5]:
#ML_TEST_WEEK2

test_data = erk.get_df(filename=config.fileName,format=config.dateFormat, start_period='2000', mode=None, to_per=True)
test_data.columns = test_data.columns.str.replace(' ', '_')

In [6]:
explanatory_df_test = test_data[config.factorName]
dependent_df_comm = test_data[config.asset_class_user]
regression_result_test = erk.regress(dependent_df_comm, explanatory_df_test, excess_mkt=False, rfcol=config.rfcol)

In [8]:
regression_result_test.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Commodities   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     24.10
Date:                Wed, 05 Aug 2020   Prob (F-statistic):           2.69e-19
Time:                        18:36:40   Log-Likelihood:                 339.10
No. Observations:                 225   AIC:                            -666.2
Df Residuals:                     219   BIC:                            -645.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3646      0.109      3.356      0.001       0.150       0.579
US_Treasuries           -0.5526      0.233     -2.376      0.018      -1.011      -0.094
Bond_Risk_Premium       -0.2268      0.198     -1.145      0.253      -0.617       0.163
Inflation_Protection     1.8561      0.348      5.340      0.000       1.171       2.541
Currency_Protection     -0.9864      0.334     -2.952      0.004      -1.645      -0.328
Alpha                    0.0031      0.004      0.806      0.421      -0.004       0.011
==============================================================================
Omnibus:                        0.828   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.661   Jarque-Bera (JB):                0.537
Skew:                           0.074   Prob(JB):                        0.764
Kurtosis:                       3.188   Cond. No.                         105.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
cv_lasso_comm = erk.regress(dependent_df_comm, explanatory_df_test, excess_mkt=False, rfcol=config.rfcol, method='cv_lasso')

Max_alpha is : 0.0005555555555555556

        Regression method is cv_lasso
        Time period is 2000-01 - 2018-09
        Alpha is 2.9007856341189672e-05
        Lambda is 0.013053535353535352
    
          intercept  World_Equities  US_Treasuries  Bond_Risk_Premium  \
cv_lasso   0.001742        0.341915      -0.281213               -0.0   

          Inflation_Protection  Currency_Protection  
cv_lasso              1.506718            -0.747364  
